# Review
## from 02. Autograd-자동미분 ~ 
20.03.31.tue.pm 6:12

* List
    * 02. Autograd
    * 03. 신경망(Neural Networks)

<code>autograd</code>패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공한다.<br>
define-by-run 프레임워크로, 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다 달라진다.

In [1]:
import torch

### Tensor

tensor를 생성하고, <code>requires_grad=True</code>를 설정하여 연산을 기록한다.

In [2]:
x = torch.ones(2, 2, requires_grad=True)  # requires_grad=True로 설정하여 연산을 기록한다.
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
# tensor에 연산을 수행한다.

y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y는 연산 결과로 생성된 것이므로 <code>.grad_fn</code>을 갖는다.

In [4]:
print(y.grad_fn)

<code>.grad_fn</code>속성은 <code>Tensor</code>를 생성한 <code>Function</code>을 참조하고 있다.

<code>y</code>에 다른 연산을 수행해보자

In [5]:
z = y * y * 3
out = z.mean()

print(z, out)  # z 또한 .grad_fn 속성을 가지고 있을까?

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


<code>.requires_grad_(...)</code>는 기존 Tensor의 <code>requires_grad</code>값을 바꿔치기 (in-place)하여 변경한다.<br>
(기존 requires_grad였던 x를 바꿔치기 한다는 의미인가?)<br>


a라는 새로운 tensor를 만들어보자

In [6]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)

False


<code>a.requires_grad</code>를 출력해보면 False로 출력되는 것을 볼 수 있다.<br>
<code>requires_grad</code>를 True로 변경하기 위해 <code>.requires_grad_(...)</code>를 사용하여 기존 값을 in-place한다.

In [7]:
a.requires_grad_(True)  # .requires_grad_(True)로 인해 기존의 requires_grad=False가 True로 바뀌었다.
print(a)

tensor([[-0.9001,  1.1225],
        [ 4.2981, -0.4974]], requires_grad=True)


<code>requires_grad</code>가 True로 변경된 것을 볼 수 있다.

In [8]:
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)  # b의 grad_fn은 Tensor를 생성한 Function을 참조한다. 

True


### Gradient 

이제 역전파(backprop)도 해보자.
앞서 정의한 변수 <code>out</code>은 하나의 스칼라 값만 갖고 있기 때문에 <code>out.backward()</code>는 <code>out.backward(torch.tensor(1.))</code>과 동일하다.<br>
(out은 z의 평균값으로 위에서 구현해 두었다) 

In [9]:
out.backward()

변화도 d(out)/dx를 출력한다.

In [10]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


벡터-야코비안 곱의 예제를 살펴보도록 하자.

* 일반적으로 , <code>torch.autograd</code>는 벡터-야코비안 곱을 계산하는 엔진이다. 

    즉, 어떤 벡터 v=(v1v2⋯vm)T 에 대해 vT⋅J 을 연산합니다. 만약 v 가 스칼라 함수 l=g(y⃗ ) 의 기울기인 경우, v=(∂l∂y1⋯∂l∂ym)T 이며, 연쇄법칙(chain rule)에 따라 벡터-야코비안 곱은 x⃗  에 대한 l 의 기울기가 됩니다:

In [11]:
x = torch.randn(3, requires_grad=True)
print('x: ', x)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2
    
print('y: ', y)

x:  tensor([0.4175, 1.4641, 0.6905], requires_grad=True)
y:  tensor([ 427.5493, 1499.2018,  707.1223], grad_fn=<MulBackward0>)


<code>torch.autograd</code>는 전체 야코비안을 직접 계산할수는 없지만, 벡터-야코비안 곱은 간단히 <code>backward</code>에 해당 벡터를 인자로 제공하여 얻을 수 있다.

In [12]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [13]:
print(y.grad)

None


In [14]:
print(v.grad)

None


Q. Jacobian-Matrix란?

* cs231n에서도 매우 혼동되었던 개념!!  - 20.03.31.Tue - 
    * <code>채워넣기</code>


In [16]:
print(x.requires_grad)
print((x ** 2).requires_grad)

True
True


In [17]:
with torch.no_grad():
    print((x ** 2).requires_grad)

False


In [18]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)


## 03. 신경망 

신경망은 <code>torch.nn</code>패키지를 사용하여 생성할 수 있다.

<code>nn</code>은 모델을 정의하고 미분하는데 <code>autograd</code>를 사용한다. <code>nn.Module</code>은 계층(layer)과 <code>output</code>을 반환하는 <code>forward(input)</code>메서드를 포함하고 있다.

이번 절에서는...<br>
간단한 <code>순전파 네트워크(Feed-forward network)</code>를 구현한다. 입력(input)을 받아 여러 계층에 차례로 전달한 후, 최종 출력(output)을 제공한다.

* 신경망의 일반적인 학습과정
    * <code>채워넣기</code>
    * 학습 가능한 매개변수 (또는 가중치(weight))를 갖는 신경망을 정의한다.
    * 데이터셋(dataset) 입력을 반복한다.
    * 입력을 신경망에서 전파(process)한다.
    * 손실(loss; 출력이 정답으로부터 얼마나 떨어져 있는지)을 계산한다. 
    * 변화도(gradient)를 신경망의 매개변수들에 역으로 전파한다.
    * 신경망의 가중치를 갱신한다. 일반적으로 다음의 간단한 규칙을 사용한다.
    * <code>가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)</code>

In [22]:
import torch
import torch.nn as  nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channer, 6 output channels, 3x3 square convolution
        # kernel
        
        # 참고 https://github.com/inmoonlight/PyTorchTutorial/blob/master/01_CNN.ipynb
        
                                         # 희한하네 
        self.conv1 = nn.Conv2d(1, 6, 3)  # Conv2d 통과 후 receptive field 값.
                                         # 초기 값이 그림과 같이 28*28일 때,  6@26*26
                                         # 튜토리얼이라 그런지 pool레이어가 생략되어있다(여기서 혼동이 온듯)
                                         # 6@13*13
        self.conv2 = nn.Conv2d(6, 16, 3) # 16@11*11
                                         # 16@6*6
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6 * 6 from Image dimension # ???
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
                             # batch 차원을 제외한 차원들의 리스트
        num_features = 1  # 일종의 flat feature 초기화 같은
        for s in size:
            num_flat_features *= s
        return num_flat_features

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    # 3개의 메서드(method)를 생성한다.
    # __init__(), forward(), num_flat_features()
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wb + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)   # 6*6 from image dimension(receptive field)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    

In [39]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


<code>forward</code>함수만 정의하고 나면, (변화도를 계산하는) <code>backward</code>함수는 <code>autograd</code>를 사용하여 자동으로 정의된다. <code>forward</code>함수에서는 어떠한 Tensor 연산을 사용해도 된다.

모델의 학습 가능한 매개변수들은 <code>net.parameters()</code>에 의해 반환된다.

In [40]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


임의의 32x32 입력값을 넣어본다.<br>
**Note** <br>
이 신경망(LeNet)의 예상되는 입력의 크기는 32x32이다.<br>
이 신경망에 MNIST 데이터셋을 사용하기 위해서는 데이터셋의 이미지 크기를 32x32로 변경해야 한다.

📌 그러면 위에서 self.f1 = nn.Linear(16 * 6 * 6, 120)의 6 * 6은 대체 뭐야??<br>
Input Image -> Conv -> ReLU -> Conv -> ReLU -> <code>fc1</code> 이 때, receptive field의 크기

In [41]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0839, -0.0825,  0.0231, -0.0565, -0.0311, -0.0410, -0.0039,  0.0912,
          0.0668,  0.0066]], grad_fn=<AddmmBackward>)


모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고, 무작위 값으로 역전파를 한다.

In [42]:
net.zero_grad()
out.backward(torch.randn(1, 10))

* Note
    * <code>torch.nn</code>은 미니-배치(mini-batch)만 지원한다.
    * 즉 하나의 샘플이 아닌, 샘플들의 미니-배치만을 입력으로 받는다.
    * 예를 들어, <code>nnConv2D는</code> <code>nSamples x nChannels x Height x Width</code> 의 4차원 Tensor를 입력으로 합니다.
    * 만약 하나의 샘플만 있다면, <code>input.unsqueeze(0)</code> 을 사용해서 가짜 차원을 추가합니다.

🔖 더 살펴볼 내용은 다음과 같다.<br>
   * 손실을 계산하는 것
   * 신경망의 가중치를 갱신하는 것
   ---
   20.04.02.Thur.pm5:00

### 손실 함수(Loss Funciton)

In [44]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0104, grad_fn=<MseLossBackward>)


In [59]:
print(loss.grad_fn)  # MSE loss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # Relu

### 역전파(Backprop)

오차를 역전파하기 위해서는 <code>loss.backward()</code>만 해주면 된다. <br>
기존 변화도를 없애는 작업이 필요한데, 그렇지 않으면 변화도가 기존의 것에 누적되기 때문이다.

In [45]:
net.zero_grad()  # zeros the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad afer backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad afer backward
tensor([ 0.0075,  0.0045, -0.0167,  0.0021,  0.0055,  0.0044])


### 가중치 갱신
실제로 많이 사용되는 가장 단순한 갱신 규칙은 확률적 경사하강법(SGD; Stochastic Gradient Descent)이다.

<code>가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient) </code>

In [46]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

신경망을 구성할 때 다양한 갱신 규칙을 사용하고 싶을 수 있다.(e.g SGD, Nesterov-SGD, Adam, RMSProp 등...)<br>
이를 위해 <code>torch.optim</code> 라는 작은 패키지에 이러한 방법들을 모두 구현해두었다.

In [47]:
import torch.optim as optim

# Optimizer를 생성한다.
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)에서는 다음과 같다.
optimizer.zero_grad()  # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()  # Does the update

🗒 **Note**<br>
<code>optimizer.zero_grad()</code>를 사용하여 수동으로 변화도 버퍼를 0으로 설정하는 것에 유의하자. 이는 역전파(Backprop) 섹션에서 설명한 것처럼 변화도가 누적되기 때문이다.


📌**VSCode에서 한번에 구현하기**<br>
다시한번 구현하면서 학습 방법을 익히자